In [ ]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, LambdaCallback

import numpy as np
import matplotlib.pyplot as plt

# reproducibility
np.random.seed(31337)



train_data_dir = 'train'
validation_data_dir = 'valid'
nb_train_samples = 2000 #22949
nb_validation_samples = 1000 #2000
nb_epoch = 10


In [3]:
X_train=np.load('catfish-X_train.npy')
Y_train=np.load('catfish-Y_train.npy')

In [5]:
X_hard=np.load('catfish-X_hard.npy')
Y_hard=np.load('catfish-Y_hard.npy')

In [36]:
# dimensions of our images.
img_width, img_height = 227, 227

model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Convolution2D(64, 3, 3))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


# Callbacks
early_stop_cb = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

checkpoit_cb = ModelCheckpoint("catfish_convnet_simple2.h5")#, save_best_only=True)

# Print the batch number at the beginning of every batch.
batch_print_cb = LambdaCallback(on_batch_begin=lambda batch, logs: print(".",end=''), 
                                on_epoch_end=lambda batch, logs: print(batch))

# Plot the loss after every epoch.
plot_loss_cb = LambdaCallback(on_epoch_end=lambda epoch, logs: 
                              print (epoch, logs))
                              #plt.plot(np.arange(epoch), logs['loss']))


model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'],
             )

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_34 (Convolution2D) (None, 225, 225, 32)  896         convolution2d_input_12[0][0]     
____________________________________________________________________________________________________
activation_56 (Activation)       (None, 225, 225, 32)  0           convolution2d_34[0][0]           
____________________________________________________________________________________________________
maxpooling2d_34 (MaxPooling2D)   (None, 112, 112, 32)  0           activation_56[0][0]              
____________________________________________________________________________________________________
convolution2d_35 (Convolution2D) (None, 110, 110, 32)  9248        maxpooling2d_34[0][0]            
___________________________________________________________________________________________

In [ ]:
history = model.fit(
        X_train,
        Y_train,
        nb_epoch=nb_epoch,
        batch_size=100,
        validation_data=(X_hard,Y_hard),
        callbacks=[early_stop_cb, checkpoit_cb, batch_print_cb, plot_loss_cb],
        verbose=0
        )


...............................................................................................................................................................................................................................................0
0 {'acc': 0.50172165608620323, 'loss': 7.7310077063010327, 'val_acc': 0.5286677683786839, 'val_loss': 7.5969772845456411}
...............................................................................................................................................................................................................................................1
1 {'acc': 0.49941209003928727, 'loss': 7.6185187007154056, 'val_acc': 0.5286677683786839, 'val_loss': 7.5969773649566292}
...............................................................................................................................................................................................................................................2
2 {'acc': 0.51121187381783628, 'l

In [25]:
model.load_weights("catfish_convnet_simple.h5")

In [27]:
score=model.evaluate(X_hard,Y_hard, verbose=0)
print("OOS %s: %.2f%%" % (model.metrics_names[1], score[1]*100))
print("OOS %s: %.2f" % (model.metrics_names[0], score[0]))

#print("min(val los)",np.min(history.history['val_loss']))

OOS acc: 52.87%
OOS loss: 7.60


In [3]:
import numpy as np
y_test=np.array([0.0,0.01,0.02,0.03,0.04,0.1,0.2,0.5,0.9,0.96,9.97,0.98,0.99,1.0])
y_test.clip(min=0.02, max=0.98)

array([ 0.02,  0.02,  0.02,  0.03,  0.04,  0.1 ,  0.2 ,  0.5 ,  0.9 ,
        0.96,  0.98,  0.98,  0.98,  0.98])